In [2]:
import os

In [3]:
%pwd

'c:\\Users\\issac\\Documents\\Work\\CNN\\cnnClass\\research'

In [5]:

os.chdir('../')
os.listdir()

['.mlai', 'cnnClass', 'data']

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir :Path
    source_url : str
    local_data_file:Path
    unzip_dir:Path
    


In [5]:
from cnnClass.constants import *
from cnnClass.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [7]:
import os
import urllib.request as request
import zipfile
from cnnClass import logger
from cnnClass.utils.common import get_size

In [8]:
class DataIngestion:

    def __init__(self,config=DataIngestionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! header :\n{headers}")
        else:
            logger.info(f"File already exists : {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-09-19 11:19:35,869: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-19 11:19:35,870: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-19 11:19:35,871: INFO: common: created directory at: artifacts]
[2024-09-19 11:19:35,871: INFO: common: created directory at: artifacts/data_ingestion]
[2024-09-19 11:19:38,823: INFO: 264058376: artifacts/data_ingestion/data.zip downloaded! header :
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4C05:3143F7:10A527B:1388419:66EBBAF4
Accept-Ranges: bytes
Date: Thu, 19 Sep 2024 05:49:36 GMT
Via: 1.1 varnish
X-Served-By: cache-hnd18748-HND
X-Cache: HIT
X-Cach